In [1]:
import requests
from io import StringIO

# скачиваем данные и сохраняем их в память
response = requests.get('https://vk.com/doc9032467_464972815?hash=2c772f61730eafb450&dl=85218bc0cf9db73adc')
data = StringIO()
data.write(response.text)


6039488

In [2]:
import pandas as pd

data.seek(0)

# обработка csv файла (табуляция выступает в качестве разделителя)
good = pd.read_csv(data, sep='\t')
# вывод 3 случайных строк данных
good.sample(3)

,context_2,context_1,context_0,reply,label
38301,NaN,NaN,"нет , юный чародей , ведь всё , что вы хотели ...",как красиво,good
55539,"открывай , это твои родители .",что ты там делаешь ?,открывай !,"открывай , мать твою !",good
44944,NaN,сними шляпу .,расслабься .,нет .,neutral


In [3]:
# поиск ответов на вопрос
rep = good[good.context_0 == 'зачем мне его принимать ?'].reply
print(rep)

13482                                затем , что это сэм .
13483    затем , что ты поимел секретаршу и обязан мне ...
13484    затем , что если ты не будешь их ставить менед...
13485                 потому что , он просил вас об этом !
13486                            потому что он любит вас .
Name: reply, dtype: object


In [4]:
# немного машинного обучения: scikit-learn пакетик
from sklearn.feature_extraction.text import TfidfVectorizer

# создаём матрицу из наших данных
# представляем каждый вопрос как вектор, где для каждого слова соответствует количество повторений в вопросе
vectorizer = TfidfVectorizer()
vectorizer.fit(good.context_0)  # находим частоту каждого слова
matrix_big = vectorizer.transform(good.context_0)  # создаём матрицу
print(matrix_big.shape)  # очень большая матрица

(60049, 14123)


In [5]:
# метод клавных компонент или сокращение матрицы
from sklearn.decomposition import TruncatedSVD

# сжимаем матрицу до 1000 столбцов
svd = TruncatedSVD(n_components=1000)
svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)
print(matrix_small.shape)
print(svd.explained_variance_ratio_.sum())  # сколько информации сохранилось

(60049, 1000)
0.6158311534393656


In [6]:
# поиск ближайшего соседа
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
import numpy as np
from functools import partial

# творим магию
def predict(self, X):
    _, indices = self.kneighbors(X, n_neighbors=10)
    result = []
    y = np.array(good.reply)
    for index in indices:
        result.append(np.random.choice(index))
    
    return y[result]

ns = KNeighborsClassifier(n_neighbors=100)
ns.fit(matrix_small, good.reply)
ns.predict = partial(predict, ns)
pipeline = make_pipeline(vectorizer, svd, ns)

In [7]:
question = input('Вопрос: ')  # вопрос

rep = good[good.context_0 == question].reply  # поиск ответов на вопрос в исходных данных
if rep.shape[0] > 0:
    print(rep)
else:
    print('Вопрос не найден')

pipeline.predict([question])[0]  # поиск ближайшего соседа

Вопрос:  ты кто ?


Вопрос не найден


'я это я .'